This notebook compares ARIMA vs Holt winters on price and demand forecasting 

In [49]:
import pandas as pd
df = pd.read_csv('/Users/suchirnaik/Downloads/merged_dataset.csv')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os



# Convert all columns to float where possible, except 'Country' and 'Year'
for col in df.columns:
    if col not in ['Country', 'Year']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Group by 'Year' and calculate mean for each column
yearly_means = df.groupby('Year').mean()

# Calculate the correlation matrix for the yearly means
correlation_matrix = yearly_means.corr()

threshold = 0.4

# Create a list to store the selected feature names
selected_features = []

# Iterate through the columns and check the correlation with 'Spot_Price_72'
for column in correlation_matrix.columns:
    if column != "Spot_Price_72":  # Exclude self-correlation
        correlation = abs(correlation_matrix["Spot_Price_72"][column])
        if correlation >= threshold:
            selected_features.append(column)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
# Preprocessing function
def preprocess_feature(df, column):
    df[column] = pd.to_numeric(df[column].str.replace(',', ''), errors='coerce')
    df[column].fillna(method='ffill', inplace=True)
    df[column].fillna(method='bfill', inplace=True)
    return df

def preprocess_feature(df, column):
    # Check if the column is of object type (likely string)
    if df[column].dtype == 'object':
        # Replace commas and convert to numeric, handling 'na' or other non-numeric values
        df[column] = pd.to_numeric(df[column].str.replace(',', ''), errors='coerce')

    # Forward fill followed by backward fill for missing values
    df[column].fillna(method='ffill', inplace=True)
    df[column].fillna(method='bfill', inplace=True)

    return df
data = df[['Country','Year','Spot_Price_72','Demand',
 'Imports',
 'Production',
 'Oil_demand_46',
 'Oil_exports_dest_51',
 'Refinery_capacity_41',
 'Refinery_throughput_43',
 'World_Production']]

selected_features = ['Spot_Price_72','Demand',
 'Imports',
 'Production',
 'Oil_demand_46',
 'Oil_exports_dest_51',
 'Refinery_capacity_41',
 'Refinery_throughput_43',
 'World_Production']

for feature in selected_features:
    data = preprocess_feature(data, feature)

# Differencing the series
data['Spot_Price_72_diff'] = data['Spot_Price_72'].diff().dropna()

data = data[data['Year'] >= 2000]




/var/folders/0w/78_w9pm16ts4trsks5rjpd440000gn/T/ipykernel_45486/2418108422.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column].fillna(method='ffill', inplace=True)
/var/folders/0w/78_w9pm16ts4trsks5rjpd440000gn/T/ipykernel_45486/2418108422.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column].fillna(method='bfill', inplace=True)
/var/folders/0w/78_w9pm16ts4trsks5rjpd440000gn/T/ipykernel_45486/2418108422.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [18]:
data

,Country,Year,Spot_Price_72,Demand,Imports,Production,Oil_demand_46,Oil_exports_dest_51,Refinery_capacity_41,Refinery_throughput_43,World_Production
182,Algeria,1996,21.33,NaN,NaN,NaN,177.0,NaN,462.0,387.0,NaN
183,Angola,1996,NaN,560.0,NaN,560.0,35.0,NaN,47.0,42.0,560.0
184,Congo,1996,NaN,0.0,NaN,0.0,2.0,NaN,21.0,10.0,0.0
185,Equatorial Guinea,1996,NaN,0.0,NaN,0.0,1.0,NaN,NaN,NaN,0.0
186,Gabon,1996,NaN,100.0,NaN,100.0,15.0,NaN,25.0,19.0,100.0
187,IR Iran,1996,18.49,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
188,Iraq,1996,NaN,NaN,0.0,NaN,387.0,NaN,598.0,419.0,NaN
189,Kuwait,1996,18.41,NaN,0.0,NaN,158.0,NaN,797.0,723.0,NaN
190,Libya,1996,20.70,NaN,NaN,NaN,200.0,NaN,565.0,299.0,NaN
191,Nigeria,1996,21.24,NaN,NaN,NaN,209.0,NaN,424.0,247.0,NaN


In [ ]:
### Univariate Arima - uses spot price only to make the prediction

In [20]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error



# Assuming 'data' is a DataFrame that includes 'Country' and 'Spot_Price_72' columns

# Dictionary to store the results
results = {}

# Loop through each country
for country in data['Country'].unique():
    # Filter the data for the current country
    country_data = data[data['Country'] == country]['Spot_Price_72'].dropna().astype(float)
    
    # Define the train-test split
    split_point = int(len(country_data) * 0.8)
    train, test = country_data[:split_point], country_data[split_point:]
    
    # Fit the ARIMA model
    model = ARIMA(train, order=(2, 1, 2))
    model_fit = model.fit()
    
    # Forecast
    forecast = model_fit.get_forecast(steps=1)
    next_year_value = forecast.predicted_mean.iloc[0]
    
    # Calculating MSE for the test dataset
    test_forecast = model_fit.get_forecast(steps=len(test))
    mse = mean_squared_error(test, test_forecast.predicted_mean)
    
    # Store results
    results[country] = {'MSE': mse, 'Next Year Forecast': next_year_value}

# Output the results
for country, metrics in results.items():
    print(f"{country} - MSE: {metrics['MSE']}, Forecast for next year: {metrics['Next Year Forecast']}")


/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.p

/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/suchirnaik/opt/anacon

Algeria - MSE: 640.1228393594113, Forecast for next year: 54.965359333245715
Angola - MSE: 623.4820038993532, Forecast for next year: 55.14155180536859
Congo - MSE: 447.62966206936744, Forecast for next year: 53.261726498226714
Equatorial Guinea - MSE: 983.621366527068, Forecast for next year: 41.89578189902228
Gabon - MSE: 570.8567653706639, Forecast for next year: 54.9253768943981
IR Iran - MSE: 530.2635392965506, Forecast for next year: 55.75526078036046
Iraq - MSE: 572.9325996505846, Forecast for next year: 52.40499455538442
Kuwait - MSE: 1030.849774681214, Forecast for next year: 42.45741227280267
Libya - MSE: 591.9983030052614, Forecast for next year: 54.368651917557045
Nigeria - MSE: 613.7514317003704, Forecast for next year: 55.78445566419698
Saudi Arabia - MSE: 644.5353587104486, Forecast for next year: 53.007157796993226
United Arab Emirates - MSE: 537.6767393093766, Forecast for next year: 54.95355192750957
Venezuela - MSE: 427.43261692224513, Forecast for next year: 39.8629

/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/suchirnaik/opt/anaconda3/lib/python3.9/si

In [ ]:
### Multivariate ARIMA  - uses additional features apart from spot price which are 'Demand', 'Imports', 'Production', 'Oil_demand_46', 'Oil_exports_dest_51', 'Refinery_capacity_41', 'Refinery_throughput_43', 'World_Production'

In [50]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

results = {}

# Loop through each country
for country in df['Country'].unique():
    # Filter the data for the current country
    country_data = data[data['Country'] == country][
        ['Year', 'Spot_Price_72', 'Demand', 'Imports', 'Production', 
         'Oil_demand_46', 'Oil_exports_dest_51', 'Refinery_capacity_41', 
         'Refinery_throughput_43', 'World_Production']
    ].dropna()
    
    # Define the train-test split
    split_point = int(len(country_data) * 0.8)
    train, test = country_data[:split_point], country_data[split_point:]
    
    # Define the features (exogenous variables)
    features = ['Demand', 'Imports', 'Production', 'Oil_demand_46', 
                'Oil_exports_dest_51', 'Refinery_capacity_41', 
                'Refinery_throughput_43', 'World_Production']
    
    # Fit the ARIMA model to predict 'Spot_Price_72'
    model = ARIMA(train['Spot_Price_72'], exog=train[features], order=(2, 1, 2))
    model_fit = model.fit()
    
    # Forecast for the next year
    forecast = model_fit.get_forecast(steps=1, exog=test[features].iloc[-1:].values)
    next_year_price = forecast.predicted_mean.iloc[0]
    
    # Calculating MSE for the test dataset
    test_forecast = model_fit.get_forecast(steps=len(test), exog=test[features].values)
    mse = mean_squared_error(test['Spot_Price_72'], test_forecast.predicted_mean)
    
    # Store results
    results[country] = {'MSE': mse, 'Next Year Price Forecast': next_year_price}

# Output the results
for country, metrics in results.items():
    print(f"{country} - MSE: {metrics['MSE']}, Forecast for next year price: {metrics['Next Year Price Forecast']}")


/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.p

/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/suchirnaik/opt/anacon

Algeria - MSE: 3953.85261553868, Forecast for next year price: 136.30593106615422
Angola - MSE: 165.31875339994914, Forecast for next year price: 103.34904780995765
Congo - MSE: 906.1294000413224, Forecast for next year price: 94.33218998134666
Equatorial Guinea - MSE: 398.96715936080085, Forecast for next year price: 62.07245821417527
Gabon - MSE: 4574.319516934249, Forecast for next year price: 85.06579833301925
IR Iran - MSE: 2050.0396835476618, Forecast for next year price: 54.41237579466835
Iraq - MSE: 2012.4302303162324, Forecast for next year price: 7.53723123525554
Kuwait - MSE: 21019.76443312194, Forecast for next year price: -32.68553370122133
Libya - MSE: 32775.58756846611, Forecast for next year price: 9.993881249816411
Nigeria - MSE: 172334.06835011064, Forecast for next year price: -450.38380558336667
Saudi Arabia - MSE: 102785.2100471649, Forecast for next year price: -309.2671738937006
United Arab Emirates - MSE: 16161.428075021307, Forecast for next year price: 1.22563

/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/suchirnaik/opt/anacon

Univariate ARIMA performs better. Just using spot price as the feature works best. This is why we adopt this methodology for all the models that is just use spot price as the feature

### Holt winter linear model 

In [22]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error



# Dictionary to store the results for each country
holt_winters_results = {}

# Loop through each country in the dataset
for country in data['Country'].unique():
    # Filter the data for the current country
    country_data = data[data['Country'] == country]['Spot_Price_72'].dropna().astype(float)
    
    # Define the train-test split
    split_point = int(len(country_data) * 0.8)
    train, test = country_data[:split_point], country_data[split_point:]
    
    # Build and fit Holt's Linear Trend model on the training data
    holt_linear_model = ExponentialSmoothing(train, trend='add', seasonal=None, damped_trend=False)
    holt_linear_model_fit = holt_linear_model.fit(optimized=True)
    
    # Forecast the next year's value after the last year in the training data
    next_year_forecast = holt_linear_model_fit.forecast(steps=1).iloc[0]
    
    # Forecasting on the test data for MSE calculation
    holt_linear_forecast = holt_linear_model_fit.forecast(steps=len(test))
    
    # Calculate MSE for the forecast
    holt_linear_mse = mean_squared_error(test, holt_linear_forecast)
    
    # Store results
    holt_winters_results[country] = {
        'MSE': holt_linear_mse,
        'Next Year Forecast': next_year_forecast
    }

# Output the results for each country
for country, results in holt_winters_results.items():
    print(f"{country} - MSE: {results['MSE']}, Forecast for next year: {results['Next Year Forecast']}")


Algeria - MSE: 502.5055122054484, Forecast for next year: 55.61319521861567
Angola - MSE: 486.9334910068502, Forecast for next year: 55.99183189912597
Congo - MSE: 373.0390366579165, Forecast for next year: 54.181981587865195
Equatorial Guinea - MSE: 490.4751771570233, Forecast for next year: 55.52647610898461
Gabon - MSE: 470.5178097275792, Forecast for next year: 54.59042956161104
IR Iran - MSE: 461.09499568342073, Forecast for next year: 53.22115545882386
Iraq - MSE: 415.1322806538009, Forecast for next year: 53.45529384654771
Kuwait - MSE: 497.0030842424232, Forecast for next year: 53.118131638424344
Libya - MSE: 484.10440500990865, Forecast for next year: 54.24294958946875
Nigeria - MSE: 489.14207887403126, Forecast for next year: 56.08293934423743
Saudi Arabia - MSE: 490.91409805996284, Forecast for next year: 54.10580976513519
United Arab Emirates - MSE: 389.84422246212534, Forecast for next year: 56.423304249696976
Venezuela - MSE: 268.8883965181588, Forecast for next year: 48.

/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/suchirnaik/opt/anaconda3/lib/python3.9/si

### Demand forecasting

In [23]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

# Load the dataset
df = pd.read_csv('/Users/suchirnaik/Downloads/merged_dataset.csv')


In [24]:
# Preprocessing function
def preprocess_feature(df, column):
    # Check if the column is of object type (likely string)
    if df[column].dtype == 'object':
        # Replace commas and convert to numeric, handling 'na' or other non-numeric values
        df[column] = pd.to_numeric(df[column].str.replace(',', ''), errors='coerce')

    # Forward fill followed by backward fill for missing values
    df[column].fillna(method='ffill', inplace=True)
    df[column].fillna(method='bfill', inplace=True)

    return df

# Select relevant columns
selected_features = ['Spot_Price_72', 'Demand']

for feature in selected_features:
    df = preprocess_feature(df, feature)

# Filter the data for the desired years
df = df[df['Year'] >= 2000]


### Arima model

In [26]:
results = {}

# Loop through each country
for country in df['Country'].unique():
    # Filter the data for the current country
    country_data = df[df['Country'] == country][['Year', 'Spot_Price_72', 'Demand']].dropna()
    
    # Define the train-test split
    split_point = int(len(country_data) * 0.8)
    train, test = country_data[:split_point], country_data[split_point:]
    
    # Fit the ARIMA model
    model = ARIMA(train['Demand'], exog=train[['Spot_Price_72']], order=(2, 1, 2))
    model_fit = model.fit()
    
    # Forecast
    forecast = model_fit.get_forecast(steps=1, exog=test[['Spot_Price_72']].iloc[-1:].values)
    next_year_demand = forecast.predicted_mean.iloc[0]
    
    # Calculating MSE for the test dataset
    test_forecast = model_fit.get_forecast(steps=len(test), exog=test[['Spot_Price_72']].values)
    mse = mean_squared_error(test['Demand'], test_forecast.predicted_mean)
    
    # Store results
    results[country] = {'MSE': mse, 'Next Year Demand Forecast': next_year_demand}

# Output the results
for country, metrics in results.items():
    print(f"{country} - MSE: {metrics['MSE']}, Forecast for next year demand: {metrics['Next Year Demand Forecast']}")

/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.p

/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/suchirnaik/opt/anacon

Algeria - MSE: 14534668.353100192, Forecast for next year demand: 42286.83879731985
Angola - MSE: 14520.269368118137, Forecast for next year demand: 835.6903421861157
Congo - MSE: 20739.26484044661, Forecast for next year demand: 548.9635304963911
Equatorial Guinea - MSE: 291702.2746049235, Forecast for next year demand: 1415.337773267248
Gabon - MSE: 128746.51467129802, Forecast for next year demand: 264.9043329920831
IR Iran - MSE: 53225343.93315171, Forecast for next year demand: 223446.10801879334
Iraq - MSE: 116635368.6195369, Forecast for next year demand: 9884.918678181564
Kuwait - MSE: 11848899.730558041, Forecast for next year demand: 22552.66854190934
Libya - MSE: 11090830.73666225, Forecast for next year demand: 5906.407139194393
Nigeria - MSE: 5945454.100808013, Forecast for next year demand: 12082.972092808574
Saudi Arabia - MSE: 84563373.68395786, Forecast for next year demand: 123287.11116434925
United Arab Emirates - MSE: 49417202.72915474, Forecast for next year demand

/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [44]:
df.dtypes

Country                        object
Year                            int64
Spot_Price_72                 float64
Crude_oil_exports_52           object
Demand                        float64
Exports                        object
Gas_Reserve                    object
Imports                        object
Production                     object
Oil_demand_46                  object
Oil_exports_dest_51            object
Refinery_capacity_41           object
Refinery_throughput_43         object
Tax                           float64
Tax_To_CIF_Ratio              float64
World_Production               object
ActiveRigs-table3.2            object
AccountBalance-table2.7        object
WellsCompleted-table3.3        object
Population-table2.1            object
petroleumExport-table2.5       object
GDP-table2.2                   object
GDPGrowthRate-table2.3         object
ExportValue-table2.4           object
crudeOilReserve-table3.1       object
CrudeOilProdution-table3.5     object
bioFuelProdu

In [48]:
country_data

,Year,Spot_Price_72,Demand,Imports,Production,Oil_demand_46,Oil_exports_dest_51,Refinery_capacity_41,Refinery_throughput_43,World_Production
418,2012,100.06,24606.0,1877.0,22726.0,786.0,1724.8,2047.0,1316.0,22726.0
432,2013,96.66,23643.0,1821.0,21820.0,831.0,1528.0,2047.0,1394.0,21820.0
446,2014,86.88,23720.0,950.0,21878.0,752.0,1964.9,2276.0,1352.0,21878.0
460,2015,41.11,24076.0,400.0,26004.0,658.0,1974.2,2276.0,1245.0,26004.0
474,2016,34.02,23715.0,0.0,27718.0,566.0,1835.0,2276.0,1035.0,27718.0
488,2017,47.63,26204.0,0.0,29793.0,503.0,1596.4,2276.0,956.0,29793.0
502,2018,64.47,20117.0,0.0,24785.0,403.0,1273.0,2276.0,842.0,24785.0
516,2019,54.04,17550.0,0.0,20559.0,367.0,847.0,2276.0,818.0,20559.0
530,2020,28.12,13725.0,0.0,18026.0,188.0,487.0,2276.0,345.0,18026.0
544,2021,51.45,13650.0,0.0,23720.0,232.0,448.0,2276.0,521.0,23720.0


### Holt winters

In [29]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error

# Assuming 'data' is a DataFrame that includes 'Country', 'Year', 'Spot_Price_72', and 'Demand' columns

# Dictionary to store the results
holt_winters_results = {}

# Loop through each country in the dataset
for country in data['Country'].unique():
    # Filter the data for the current country
    country_data = data[data['Country'] == country][['Year', 'Spot_Price_72', 'Demand']].dropna()
    
    # Define the train-test split
    split_point = int(len(country_data) * 0.8)
    train, test = country_data[:split_point], country_data[split_point:]
    
    # Build and fit Holt-Winters Exponential Smoothing model on the training data
    holt_winters_model = ExponentialSmoothing(train['Demand'], trend='add', seasonal=None, damped_trend=False)
    holt_winters_model_fit = holt_winters_model.fit(optimized=True)
    
    # Forecast the next year's demand after the last year in the training data
    next_year_demand_forecast = holt_winters_model_fit.forecast(steps=1)
    
    # Forecasting on the test data for MSE calculation
    holt_winters_demand_forecast = holt_winters_model_fit.forecast(steps=len(test))
    
    # Calculate MSE for the demand forecast
    holt_winters_mse = mean_squared_error(test['Demand'], holt_winters_demand_forecast)
    
    # Store results
    holt_winters_results[country] = {
        'MSE': holt_winters_mse,
        'Next Year Demand Forecast': next_year_demand_forecast.iloc[0]
    }

# Output the results for each country
for country, results in holt_winters_results.items():
    print(f"{country} - MSE: {results['MSE']}, Forecast for next year demand: {results['Next Year Demand Forecast']}")


Algeria - MSE: 27924.991041430796, Forecast for next year demand: 604.3976285751586
Angola - MSE: 20698.962400867098, Forecast for next year demand: 838.7743525213715
Congo - MSE: 63512.17748997665, Forecast for next year demand: 575.0024069143061
Equatorial Guinea - MSE: 65607.2954320214, Forecast for next year demand: 572.7853246426228
Gabon - MSE: 387089.97986095666, Forecast for next year demand: 263.000006496906
IR Iran - MSE: 387089.97986095666, Forecast for next year demand: 263.000006496906
Iraq - MSE: 387089.97986095666, Forecast for next year demand: 263.000006496906
Kuwait - MSE: 387089.97986095666, Forecast for next year demand: 263.000006496906
Libya - MSE: 387089.97986095666, Forecast for next year demand: 263.000006496906
Nigeria - MSE: 387089.97986095666, Forecast for next year demand: 263.000006496906
Saudi Arabia - MSE: 387089.97986095666, Forecast for next year demand: 263.000006496906
United Arab Emirates - MSE: 387089.97986095666, Forecast for next year demand: 263

/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/suchirnaik/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/suchirnaik/opt/anaconda3/lib/python3.9/si